In [2]:
import numpy as np, pandas as pd, time, os, subprocess, scipy as sp, re
import seaborn; seaborn.set_style('whitegrid')
import itertools
import matplotlib.pyplot as plt, importlib
import scipy.sparse
#import diffmap as dm#, scanpy as sc

data_pfx = '/oak/stanford/groups/akundaje/abalsubr/search_ui/avocado_model/'

# Experimental embeddings

In [3]:
a = np.load(data_pfx + 'experiment_embedding.npz')
a

FileNotFoundError: [Errno 2] No such file or directory: '/oak/stanford/groups/akundaje/abalsubr/search_ui/avocado_model/experiment_embedding.npz'

# Read avocado embeddings

In [6]:
import numpy
from tqdm import tqdm

def load_datasets(data_set, data_dir):
    data = []
    for celltype, assay in tqdm(data_set):
        if 'minus' in assay:
            continue
        filename = "{}/{}.{}.pilot.{}npz".format(data_dir, celltype, assay, 'arcsinh.' if 'ENCODE' in data_dir else '')
        track = numpy.load(filename)['arr_0'][:600000]
        if 'plus' in assay:
            filename = filename.replace("plus", "minus")
            track += numpy.load(filename)['arr_0'][:600000]
        data.append(track)
    return numpy.array(data)


experiments = numpy.loadtxt("data/ENCODE_Tracks.txt", delimiter='\t', dtype=str)
celltypes = list(numpy.unique(experiments[:,0]))
assays = list(numpy.unique(experiments[:,1]))

# real_data = load_datasets(experiments, '/net/noble/vol5/user/jmschr/proj/ENCODE/')
# imputed_data = data = load_datasets(experiments, '/tmp/jmschr/')

# real_corr = numpy.corrcoef(real_data).astype('float32')
# imputed_corr = numpy.corrcoef(imputed_data).astype('float32')

# numpy.savez_compressed("real_correlations_3150_600000.npz", real_corr)
# numpy.savez_compressed("imputed_correlations_3150_600000.npz", imputed_corr)

In [7]:
experiments

array([['J000', 'ChIP-seq_H3K79me2_signal_p-value'],
       ['J002', 'CAGE_plus_strand_signal_of_all_reads'],
       ['J006', 'polyA_RNA-seq_minus_strand_signal_of_all_reads'],
       ...,
       ['J221', 'ChIP-seq_H3K9me3_signal_p-value'],
       ['J382', 'polyA_RNA-seq_plus_strand_signal_of_all_reads'],
       ['J223', 'ChIP-seq_H3K9me3_signal_p-value']], dtype='<U55')